# Class 21:  joint entropy and the REVEAL algorithm

We'll use the bladder cancer gene expression data to test out the REVEAL algorithm. First, we'll load the data and filter to include only genes for which the median log2 expression level is > 12.  That should give us 164 genes to work with.

In [81]:
import pandas
gene_matrix_for_network_df = pandas.read_csv("shared/bladder_cancer_genes_tcga.txt", sep="\t")
gene_matrix_for_network = gene_matrix_for_network_df.as_matrix()
print(gene_matrix_for_network.shape)

(4473, 414)


Filter the matrix to include only rows for which the column-wise median is > 12; matrix should now be 164 x 414; this is not strictly necessary but will help prevent us from over-burderning the EC2 instance and it will enable us to easily compute the partial correlation matrix using the inverse of the covariance matrix. Print the size of the filtered matrix, as a sanity check.

In [82]:
import numpy
gene_matrix_np = numpy.array(gene_matrix_for_network)
genes_keep = numpy.where(numpy.median(gene_matrix_np, axis=1) > 12)
matrix_filt = gene_matrix_np[genes_keep, ][0]
matrix_filt.shape
N = matrix_filt.shape[0]
M = matrix_filt.shape[1]

Binarize the gene expression matrix using the mean value as a breakpoint, turning it into a NxM matrix of logicals (true/false).  Call it `gene_matrix_binarized`.

In [83]:
gene_matrix_binarized = numpy.tile(numpy.mean(matrix_filt, axis=1),(M,1)).transpose() < matrix_filt
print(gene_matrix_binarized.shape)

(164, 414)


The core part of the REVEAL algorithm is a function that can compute the joint entropy of a collection of binary (TRUE/FALSE) vectors X1, X2, ..., Xn (where length(X1) = length(Xi) = M).
Write a function `entropy_multiple_vecs` that takes as its input a nxM matrix (where n is the number of variables, i.e., genes, and M is the number of samples in which gene expression was measured). The function should use the log2 definition of the Shannon entropy. It should return the joint entropy H(X1, X2, ..., Xn) as a scalar numeric value. I have created a skeleton version of this function for you, in which you can fill in the code. I have also created some test code that you can use to test your function, below.

In [85]:
def entropy_multiple_vecs(binary_vecs):
    binary_vecs = gene_matrix_binarized[0:4, ]
    [n, M] = binary_vecs.shape
    binary_df = pandas.DataFrame(binary_vecs.transpose())
    binary_df_counts = binary_df.groupby(binary_df.columns.values.tolist()).size().values
    probvec = binary_df_counts/M
    hvec = -probvec*numpy.log2(probvec)
    return numpy.sum(hvec)

This test case should produce the value 3.938:

In [86]:
print(entropy_multiple_vecs(gene_matrix_binarized[0:4,]))

3.93769024041


## Example implementation of the REVEAL algorithm:

**See the example code in `class21_reveal_R.ipynb`; gold star for an enterprising student who codes this algorithm up in Python! (but please don't run it past stage 2 on the JupyterHub server!)